# Team Project (Title tbd)

## Ideas:
- Does regular season win record impact playoff performance?
    - Is a winning record enough to do well in the postseason?
- Can a player actually "carry" a team?
    - 
- 

In [24]:
import pandas as pd
import plotly as px
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as pxo

In [25]:
per = pd.read_csv("data/per.csv", index_col="Player")
ppg = pd.read_csv("data/ppg.csv")
ppg.drop(columns=["Player-additional", "Rk"], inplace=True)
per = per.rename(columns={"PER▼":"PER"})
per.drop(columns="Player-additional", inplace=True)
per = per.merge(ppg, left_on="Player", right_on="Player", how="inner")
per = per[per["Tm"] != "TOT"]
per

,Player,Rk,Pos,Age,Tm,PER,3PAr,PTS
0,Nikola Jokić,1,C,27,DEN,31.5,0.146,24.5
1,Joel Embiid,2,C,28,PHI,31.4,0.151,33.1
2,Giannis Antetokounmpo,3,PF,28,MIL,29.0,0.134,31.1
3,Luka Dončić,4,PG,23,DAL,28.7,0.373,32.4
4,Anthony Davis,5,C,29,LAL,27.8,0.077,25.9
...,...,...,...,...,...,...,...,...
217,Dillon Brooks,184,SF,27,MEM,9.4,0.443,14.3
221,Royce O'Neale,186,SF,29,BRK,9.2,0.707,8.8
222,Gabe Vincent,187,PG,26,MIA,9.0,0.617,9.4
226,Reggie Bullock,189,SF,31,DAL,7.7,0.845,7.2


In [26]:
three_pt_atmpts = pd.read_csv("data/3pa.csv", index_col="Team")
three_pt_atmpts.rename(index=lambda s: s[:-1] if s.endswith("*") else s, inplace=True)
three_pt_atmpts

,Rk,G,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,AST,TOV,PTS
Team,,,,,,,,,,,,,,,,,
Sacramento Kings,1.0,82,43.6,88.2,0.494,13.8,37.3,0.369,29.8,50.9,0.586,19.8,25.1,0.790,27.3,13.5,120.7
Golden State Warriors,2.0,82,43.1,90.2,0.479,16.6,43.2,0.385,26.5,47.0,0.564,16.0,20.2,0.794,29.8,16.3,118.9
Atlanta Hawks,3.0,82,44.6,92.4,0.483,10.8,30.5,0.352,33.9,61.8,0.548,18.5,22.6,0.818,25.0,12.9,118.4
Boston Celtics,4.0,82,42.2,88.8,0.475,16.0,42.6,0.377,26.2,46.2,0.567,17.5,21.6,0.812,26.7,13.4,117.9
Oklahoma City Thunder,5.0,82,43.1,92.6,0.465,12.1,34.1,0.356,31.0,58.5,0.530,19.2,23.7,0.809,24.4,13.0,117.5
Los Angeles Lakers,6.0,82,42.9,89.0,0.482,10.8,31.2,0.346,32.1,57.8,0.555,20.6,26.6,0.775,25.3,14.1,117.2
Utah Jazz,7.0,82,42.5,89.8,0.473,13.3,37.8,0.353,29.2,52.0,0.560,18.7,23.8,0.786,26.0,15.4,117.1
Memphis Grizzlies,8.0,82,43.7,92.1,0.475,12.0,34.2,0.351,31.7,57.9,0.548,17.5,23.8,0.733,26.0,13.6,116.9
Milwaukee Bucks,9.0,82,42.7,90.4,0.473,14.8,40.3,0.368,27.9,50.1,0.557,16.6,22.4,0.743,25.8,14.6,116.9


In [27]:
abbreviations = pd.read_csv("data/abbreviations.csv")
teams = pd.read_csv("data/teams.csv")

# Rename index and columns to preferred names
abbreviations.rename(columns={"prefix_1": "abbrev"}, inplace=True)

# Regularize abbreviations
abbreviations["abbrev"] = abbreviations["abbrev"].apply(lambda s: s.upper())

# Drop unneeded column
abbreviations.drop(columns="prefix_2", inplace=True)
abbreviations.loc[abbreviations["abbrev"] == "NO", "name"] = "New Orleans Pelicans"
abbreviations.loc[abbreviations["abbrev"] == "CHA", "name"] = "Charlotte Hornets"

teams = teams.merge(abbreviations, left_on="Team", right_on="name", how="outer")
teams.drop(columns=["name", "Rk"])

,Team,Overall,abbrev
0,Atlanta Hawks,41-41,ATL
1,Boston Celtics,57-25,BOS
2,Brooklyn Nets,45-37,BKN
3,Charlotte Hornets,27-55,CHA
4,Chicago Bulls,40-42,CHI
5,Cleveland Cavaliers,51-31,CLE
6,Dallas Mavericks,38-44,DAL
7,Denver Nuggets,53-29,DEN
8,Detroit Pistons,17-65,DET
9,Golden State Warriors,44-38,GS


## Question 1: How does player efficiency rating (PER) correlate with a team's win-loss record?



In [28]:
model = LinearRegression()
# display(per)

teams[["wins", "losses"]] = teams["Overall"].str.split("-", expand=True).astype(int)
# teams["W/L Ratio"] = round((teams["wins"] / (teams["losses"] + teams["wins"])), 2)
# WL = teams["W/L Ratio"]
#display(teams)

top_players = per.groupby("Tm")["PER"].max().reset_index()
top_players.loc[top_players["Tm"] == "CHO", "Tm"] = "CHA"
top_players = pd.merge(top_players, teams[["abbrev", "wins"]], left_on="Tm", right_on="abbrev")
top_players.drop(columns="abbrev", inplace=True)

required_columns = ["Tm", "PER", "wins"]
assert all(column in top_players.columns for column in required_columns), "Missing columns"

pxo.scatter(top_players, x = "PER", y = "wins", color="Tm").update_layout(xaxis_title = "Highest PER on Team", yaxis_title = "Teams Wins", title="How Does Having a High PER Player Affect Wins in a Season?")

## Question 2: What impact does age have on PPG?

In [38]:
display(per)
required_columns = ["Player", " Rk", "Pos", "Age", "Tm", "PER", "3PAr", "PTS"]
assert all(column in per.columns for column in required_columns), "Missing columns"

age_avgs = per.groupby("Age")["PTS"].mean()
age_avgs
pxo.bar(age_avgs).update_layout( yaxis_title = "Points Per Game", title="How Does Age Affect The Average Points Per Game of an NBA Player?")

,Player,Rk,Pos,Age,Tm,PER,3PAr,PTS
0,Nikola Jokić,1,C,27,DEN,31.5,0.146,24.5
1,Joel Embiid,2,C,28,PHI,31.4,0.151,33.1
2,Giannis Antetokounmpo,3,PF,28,MIL,29.0,0.134,31.1
3,Luka Dončić,4,PG,23,DAL,28.7,0.373,32.4
4,Anthony Davis,5,C,29,LAL,27.8,0.077,25.9
...,...,...,...,...,...,...,...,...
217,Dillon Brooks,184,SF,27,MEM,9.4,0.443,14.3
221,Royce O'Neale,186,SF,29,BRK,9.2,0.707,8.8
222,Gabe Vincent,187,PG,26,MIA,9.0,0.617,9.4
226,Reggie Bullock,189,SF,31,DAL,7.7,0.845,7.2


### Follow up questions:
- Could this be more indicative of the impact that ppg has on the longevity of a career?

## Question 3: Does the three-point shot frequency correlate with higher team scoring averages and how does this impact Win/Loss record?


In [41]:
ThreePointShotFreq = per["3PAr"]
# display(teams)
teamPPG = three_pt_atmpts["PTS"]
wins = teams["wins"]

team_stats = three_pt_atmpts.merge(teams, left_on="Team", right_on="Team")
team_stats["3PA"] = team_stats["3PA"]

required_columns = ["Team", "Rk_x", "G", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "2P",
                   "2PA", "2P%", "FT", "FTA", "FT%", "AST", "TOV", "PTS", "Rk_y",
                   "Overall", "name", "abbrev", "wins", "losses"]
assert all(column in team_stats.columns for column in required_columns), "Missing columns"

pxo.scatter(team_stats, x = "wins", y = "PTS", size = "3PA", color = "Team")


### Follow up questions:
- Is this more telling of the state of the league and the emphasis there is on the three-point shot?